In [1]:
# This is my personal data folder. Change to your own
dir = "/Users/piromast/Documents/Kaggle/SantanderData/"

In [244]:
# Import necessary stuff
from pandas import *
import numpy as np
import os.path
set_option("display.max_columns",50)

In [279]:
# Import data (first 1000 rows for now)
data=read_csv(os.path.join(dir,'train_ver2.csv'), nrows = 10000, skipinitialspace=True)

In [280]:
# Remove nans
#data.dropna(axis=1, how='all', inplace=True)
#data.dropna(how='any', inplace=True)
data.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [281]:
# Import description table (describing the meaning of each column)
description = read_csv(os.path.join(dir,'column_description.csv'),encoding = 'latin1')
description

,Column name,Description,"Variable type: 0: index, 1: continuous, 2: categorical, 3: date, 4: product"
0,fecha_dato,The table is partitioned for this column,0
1,ncodpers,Customer code,0
2,ind_empleado,"Employee index: A active, B ex employed, F fil...",2
3,pais_residencia,Customer's Country residence,2
4,sexo,Customer's sex,2
5,age,Age,1
6,fecha_alta,The date in which the customer became as the f...,3
7,ind_nuevo,New customer Index. 1 if the customer register...,2
8,antiguedad,Customer seniority (in months),1
9,indrel,"1 (First/Primary), 99 (Primary customer during...",2


In [282]:
# Select product column names and build indeces range
column_names = data.columns.values
product_names = [(i, s) for i, s in enumerate(column_names) if 'ind_' in s and '_ult1' in s]
product_indeces = [e[0] for e in product_names]
mInd = min(product_indeces)
MInd = max(product_indeces)
product_indeces_range = range(mInd,MInd)

In [283]:
product_names

[(24, 'ind_ahor_fin_ult1'),
 (25, 'ind_aval_fin_ult1'),
 (26, 'ind_cco_fin_ult1'),
 (27, 'ind_cder_fin_ult1'),
 (28, 'ind_cno_fin_ult1'),
 (29, 'ind_ctju_fin_ult1'),
 (30, 'ind_ctma_fin_ult1'),
 (31, 'ind_ctop_fin_ult1'),
 (32, 'ind_ctpp_fin_ult1'),
 (33, 'ind_deco_fin_ult1'),
 (34, 'ind_deme_fin_ult1'),
 (35, 'ind_dela_fin_ult1'),
 (36, 'ind_ecue_fin_ult1'),
 (37, 'ind_fond_fin_ult1'),
 (38, 'ind_hip_fin_ult1'),
 (39, 'ind_plan_fin_ult1'),
 (40, 'ind_pres_fin_ult1'),
 (41, 'ind_reca_fin_ult1'),
 (42, 'ind_tjcr_fin_ult1'),
 (43, 'ind_valo_fin_ult1'),
 (44, 'ind_viv_fin_ult1'),
 (45, 'ind_nomina_ult1'),
 (46, 'ind_nom_pens_ult1'),
 (47, 'ind_recibo_ult1')]

In [297]:
# Build a dataframe only with products
product_df = data.iloc[:,product_indeces_range]
products_array = product_df.values

In [285]:
# Build a Series with number of products and use description as indeces
desclist=description.iloc[product_indeces_range,1].tolist()
num_products = Series(product_df.sum().tolist(),index=desclist)
num_products

Saving Account                0.0
Guarantees                    0.0
Current Accounts           9296.0
Derivada Account              0.0
Payroll Account             369.0
Junior Account               67.0
Màs particular Account       12.0
particular Account            0.0
particular Plus Account       0.0
Short-term deposits           3.0
Medium-term deposits          2.0
Long-term deposits           85.0
e-account                   633.0
Funds                        22.0
Mortgage                      2.0
Pensions                      6.0
Loans                         2.0
Taxes                       106.0
Credit Card                 109.0
Securities                   32.0
Home Account                  0.0
Payroll                     218.0
Pensions                    240.0
dtype: float64

In [286]:
# Let's start scikit-learning
import sklearn.preprocessing, sklearn.pipeline

In [287]:
# Make dummy variable representation (onehot) for categorical variables
# Option 1: user label encoder on each column and onehot encoder on entire dataframe.
# This produces an encoding that will probably depend on the number of rows
pandas.options.mode.chained_assignment = None

le = sklearn.preprocessing.LabelEncoder()
oh = sklearn.preprocessing.OneHotEncoder()

# Select categorical variables
CatVariables = data.iloc[:,(description.iloc[:,2]==2).tolist()]

# Store label encoder for each column in a list (to undo encoding)
label_encoder_list = []

# Apply lable encoder to each column. Then apply onehot encoder to all the data array
for i, colname in enumerate(CatVariables):
    col = CatVariables[colname].tolist()
    
    CatVariables.iloc[:,i] = le.fit_transform(col)
    label_encoder_list.append(le)
    
OneHotFeatures = oh.fit_transform(CatVariables.values)

In [236]:
# # Make dummy variable representation (onehot) for categorical variables
# # Option 2: apply label encoder and onehot encoding on each column seperately
# # I use a pipeline here. All this doesn't work at the moment
# 
# pandas.options.mode.chained_assignment = None
# 
# le = sklearn.preprocessing.LabelEncoder()
# oh = sklearn.preprocessing.OneHotEncoder()
# encoder = sklearn.pipeline.Pipeline([('LabelEncoder',le),('OnehotEncoder',oh)])
# 
# # Select categorical variables
# CatVariables = data.iloc[:,(description.iloc[:,2]==2).tolist()]
# encoder_list = []
# for i, colname in enumerate(CatVariables):
#     le = sklearn.preprocessing.LabelEncoder()
#     oh = sklearn.preprocessing.OneHotEncoder()
#     #encoder = sklearn.pipeline.Pipeline([('LabelEncoder',le),('OnehotEncoder',oh)])
#     col = CatVariables[colname].tolist()
#     
#     CatVariables.iloc[:,i] = le.fit_transform(col)
#     
#     encoder_list.append(le)

In [312]:
# Build feature array including onehot features and continuous data
from scipy.sparse import hstack, coo_matrix
ContinuousVariables = data.iloc[:,(description.iloc[:,2]==1).tolist()].values
FeatureMatrix=hstack([OneHotFeatures, coo_matrix(ContinuousVariables.astype(np.float64))]).todense()
imputer = sklearn.preprocessing.Imputer(missing_values=0,strategy="mean",axis=0)
FeatureMatrix = imputer.fit_transform(FeatureMatrix)

/Users/piromast/Python35VirtualEnv/lib/python3.5/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)


In [326]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
single_product = products_array[:,13]
logreg.fit(FeatureMatrix,single_product)
prediction = logreg.predict(FeatureMatrix)

In [327]:
from sklearn.metrics import classification_report
classification_report(single_product, prediction)

/Users/piromast/Python35VirtualEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'             precision    recall  f1-score   support\n\n        0.0       1.00      1.00      1.00      9978\n        1.0       0.00      0.00      0.00        22\n\navg / total       1.00      1.00      1.00     10000\n'

In [ ]:
from sklearn.model_selection import train_test_split